## Evaluating a vectorial function on CPU and GPU

### CPU: plain and numpy

In [1]:
import numpy as np
from numba import njit, jit

# Python plain implementation w/ numba 
@njit
def grade2_vector(x, y, a, b, c):
    z = np.zeros(x.size)
    for i in range(x.size):
        z[i] = a*x[i]*x[i] + b*y[i] + c
    return z

# Numpy ufunc
def grade2_ufunc(x, y, a, b, c):
    return a*x**2 + b*y + c

# size of the vectors
size = 5_000_000

# allocating and populating the vectors
a_cpu = np.random.rand(size)
b_cpu = np.random.rand(size)
c_cpu = np.zeros(size)

a = 3.5
b = 2.8
c = 10

# Printing input values
#print(a_cpu)
#print(b_cpu)
# Random function in Numpy always use float64
print(a_cpu.dtype)

c_cpu = grade2_vector(a_cpu, b_cpu, a, b, c)


# Evaluating the time

# Numba Python: huge improvement, better that numpy code
%timeit -n 5 -r 2 grade2_vector(a_cpu, b_cpu, a, b, c)

# w/ a numpy ufunc manually coded
%timeit -n 5 -r 2 grade2_ufunc(a_cpu, b_cpu, a, b, c)

# using the general numpy ufunc 
%timeit -n 5 -r 2 a*a_cpu**2 + b*b_cpu + c



float64
8.55 ms ± 199 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
18.9 ms ± 22.8 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
18.9 ms ± 207 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)


### Librería Cupy

In [2]:
import cupy as cp

In [29]:
# Copy arrays from CPU to GPU
a_gpu = cp.asarray(a_cpu)
b_gpu = cp.asarray(b_cpu)

# Allocate output on GPU
c_gpu = cp.zeros_like(a_gpu)

# Define operation using CuPy ufuncs
def grade2_cupy(x, y, a, b, c):
    return a * x**2 + b * y + c

# Warm-up
grade2_cupy(a_gpu, b_gpu, a, b, c)
cp.cuda.Stream.null.synchronize()

# Benchmark INCLUDING data copy (correct)
cp.cuda.Stream.null.synchronize()
cp._default_memory_pool.free_all_blocks()

start = cp.cuda.Event()
end = cp.cuda.Event()

start.record()
a_gpu = cp.asarray(a_cpu)
b_gpu = cp.asarray(b_cpu)
c_gpu = grade2_cupy(a_gpu, b_gpu, a, b, c)
end.record()

end.synchronize()
print("CuPy (with copy) time [ms]:", 
      cp.cuda.get_elapsed_time(start, end))

CuPy (with copy) time [ms]: 15.598591804504395


In [28]:
# Create arrays directly on GPU
a_gpu = cp.random.rand(size, dtype=cp.float64)
b_gpu = cp.random.rand(size, dtype=cp.float64)

# Warm-up
grade2_cupy(a_gpu, b_gpu, a, b, c)
cp.cuda.Stream.null.synchronize()

# Benchmark without data copy
start.record()
c_gpu = grade2_cupy(a_gpu, b_gpu, a, b, c)
end.record()

end.synchronize()
print("CuPy (no copy) time [ms]:", 
      cp.cuda.get_elapsed_time(start, end))


CuPy (no copy) time [ms]: 4.786880016326904


### Librería Numba

In [30]:
from numba import vectorize

In [31]:
@vectorize(
    ['float64(float64, float64, float64, float64, float64)'],
    target='cuda'
)
def grade2_numba_gpu(x, y, a, b, c):
    return a*x*x + b*y + c


In [33]:
# Warm-up
grade2_numba_gpu(a_cpu[:10], b_cpu[:10], a, b, c)
cp.cuda.Stream.null.synchronize()

# Benchmark with implicit copy
start = cp.cuda.Event()
end = cp.cuda.Event()

start.record()
c_gpu = grade2_numba_gpu(a_cpu, b_cpu, a, b, c)
end.record()

end.synchronize()
print("Numba GPU (with copy) time [ms]:",
      cp.cuda.get_elapsed_time(start, end))

Numba GPU (with copy) time [ms]: 17.636383056640625


In [27]:
# Manual copy to GPU
a_gpu = cp.asarray(a_cpu)
b_gpu = cp.asarray(b_cpu)

# Warm-up
grade2_numba_gpu(a_gpu, b_gpu, a, b, c)
cp.cuda.Stream.null.synchronize()

# Benchmark without copy
start = cp.cuda.Event()
end = cp.cuda.Event()

start.record()
c_gpu = grade2_numba_gpu(a_gpu, b_gpu, a, b, c)
end.record()

end.synchronize()
print("Numba GPU (no copy) time [ms]:", 
      cp.cuda.get_elapsed_time(start, end))

Numba GPU (no copy) time [ms]: 1.6814080476760864


### Análisis de resultados

Los resultados obtenidos muestran claramente el impacto del uso de la GPU y, especialmente, del coste asociado a la copia de datos entre CPU y GPU.

En el caso de CuPy, cuando se incluyen las copias de datos desde la CPU a la GPU, el tiempo de ejecución aumenta de forma notable (≈15.6 ms). Sin embargo, cuando los datos se crean directamente en la GPU y se evita la transferencia, el tiempo se reduce significativamente (≈4.8 ms). Esto indica que el cálculo en sí es rápido, pero la transferencia de datos puede convertirse en el principal cuello de botella.

Para Numba en GPU, el comportamiento es similar, aunque aún más acusado. Con copia implícita de datos, el tiempo es comparable al de CuPy con copia (≈17.6 ms). No obstante, cuando los datos ya residen en la GPU, el tiempo de ejecución se reduce drásticamente hasta valores muy bajos (≈1.7 ms), siendo la opción más rápida de todas las probadas.

En conjunto, estos resultados ponen de manifiesto que el uso de la GPU ofrece mejoras significativas de rendimiento, pero solo cuando se minimiza el coste de transferencia de datos. El mayor beneficio se obtiene cuando los datos permanecen en la GPU durante todo el cálculo, lo que resalta la importancia de una gestión eficiente de la memoria en aplicaciones aceleradas por GPU.